In [3]:
# importing Library 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
import re
from dotenv import load_dotenv
import psycopg2
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [5]:
sheet_id = '13cPFTjh225ah9TbRLOVFsF_-fCUG2cr2owHp5f85YzQ'
scope = 'https://spreadsheets.google.com/feeds'
credentials = ServiceAccountCredentials.from_json_keyfile_name('./moc-trade-data-scrapper-b9a967474f65.json', scope)
worksheet = gspread.authorize(credentials).open_by_key(sheet_id).worksheet('Non_coal_auction_DB')
data = worksheet.get_all_values()

data

[['id',
  'auction_authority',
  'name_of_the_block',
  'mineral',
  'auction_date',
  'ml_or_cl',
  'area_in_hectare',
  'final_bid_in_per_cent',
  'preferred_bidder'],
 ['1',
  'Andhra Pradesh',
  'Gudipadu block',
  'Limestone',
  '6/8/2016',
  'ML',
  '40.82',
  '8.12',
  'Penna Cements'],
 ['2',
  'Andhra Pradesh',
  'Erragudi-Hussainapuram–Yanakandla',
  'Limestone',
  '9/26/2017',
  'ML',
  '131.57',
  '10.6',
  'Sree Jayajothi Cements Pvt Ltd'],
 ['3',
  'Andhra Pradesh',
  'Nandavaram–Venkatapuram',
  'Limestone',
  '9/26/2017',
  'ML',
  '24.739',
  '10.7',
  'Sree Jayajothi Cements Pvt Ltd'],
 ['4',
  'Andhra Pradesh',
  'Chigargunta – Bisanatham',
  'Gold',
  '7/27/2018',
  'ML',
  '262.01',
  '38.25',
  'NMDC'],
 ['5',
  'Andhra Pradesh',
  'Chintalayapalle-Abdullapuram-Korumanipalli (CAK)',
  'Limestone',
  '11/28/2018',
  'ML',
  '417.51',
  '13.4',
  'Adani Cementation'],
 ['6',
  'Andhra Pradesh',
  'Gorlagutta',
  'Limestone',
  '5/27/2021',
  'ML',
  '181.03',
  '7.2

In [23]:
df = pd.DataFrame(data[1:], columns=data[0])

In [24]:
df.rename(columns={'auction_authority': 'state'}, inplace=True)

In [33]:
df

,id,state,name_of_the_block,mineral,auction_date,ml_or_cl,area_in_hectare,final_bid_in_per_cent,preferred_bidder
0,1,Andhra Pradesh,Gudipadu block,Limestone,6/8/2016,ML,40.82,8.12,Penna Cements
1,2,Andhra Pradesh,Erragudi-Hussainapuram–Yanakandla,Limestone,9/26/2017,ML,131.57,10.6,Sree Jayajothi Cements Pvt Ltd
2,3,Andhra Pradesh,Nandavaram–Venkatapuram,Limestone,9/26/2017,ML,24.739,10.7,Sree Jayajothi Cements Pvt Ltd
3,4,Andhra Pradesh,Chigargunta – Bisanatham,Gold,7/27/2018,ML,262.01,38.25,NMDC
4,5,Andhra Pradesh,Chintalayapalle-Abdullapuram-Korumanipalli (CAK),Limestone,11/28/2018,ML,417.51,13.4,Adani Cementation
...,...,...,...,...,...,...,...,...,...
430,431,Central Government/ Andhra Pradesh,Balepalyam Tungsten and Associated Mineral Block,Tungsten and Associated Mineral,10/22/2024,CL,308.3,6.55,"Hindustan Zinc Limited (HZL), Vedanta"
431,432,Central Government/Arunachal Pradesh,Radhpu Graphite and Vanadium Block,Graphite and Vanadium,10/22/2024,CL,152.74,95,Orissa Metaliks Private Limited
432,433,Central Government/Arunachal Pradesh,Endolin-Isholin Graphite Block,Graphite,10/23/2024,CL,503.15,237.5,Mamco Mining Private Limited
433,434,Central Government/Karnataka,"Sanyasikoppa Cobalt, Manganese and Iron Block","Cobalt, Manganese and Iron",10/24/2024,CL,1055.64,45,Vedanta


In [32]:
# Replace empty strings in 'area_in_hectare' with NaN
df['area_in_hectare'] = df['area_in_hectare'].replace('', np.nan)

In [34]:
# Save it in postgres database

from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os
from dotenv import load_dotenv
import math

# Load environment variables
load_dotenv(override=True)

# Define the PostgreSQL connection string
connection_string = os.getenv('POSTGRES_DB_URL')

# Create the SQLAlchemy engine for PostgreSQL
engine = create_engine(connection_string)

# Define the base for the declarative mapping
Base = declarative_base()

# Define the table schema for PostgreSQL
class AuctionData(Base):
    __tablename__ = 'mom_auction'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary Key
    state = Column(String(255))
    name_of_the_block = Column(String(255))
    mineral = Column(String(255))
    auction_date = Column(Date)
    ml_or_cl = Column(String(10))
    area_in_hectare = Column(Float)
    final_bid_in_per_cent = Column(Float)
    preferred_bidder = Column(String(255))

# Create the table in the PostgreSQL database
Base.metadata.create_all(engine)

# Prepare your DataFrame (scraped data already processed in df)

# Define a session to interact with the PostgreSQL database
Session = sessionmaker(bind=engine)
session = Session()

# Convert the DataFrame to a list of dictionaries, compatible with SQLAlchemy ORM
auction_data_dicts = df.to_dict(orient='records')

# Insert data into the auction_data table
for record in auction_data_dicts:
    # Replace nan with None for PostgreSQL compatibility
    record = {key: (None if (isinstance(value, float) and math.isnan(value)) else value) 
              for key, value in record.items()}
    
    auction_entry = AuctionData(**record)  # Unpack the dictionary into the ORM object
    session.add(auction_entry)

# Commit the transaction
session.commit()

print("Data inserted successfully into PostgreSQL!")

# Close the session
session.close()

/var/folders/ln/sphck21d1ldb9687pbhzy38c0000gn/T/ipykernel_2258/1321650604.py:21: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Data inserted successfully into PostgreSQL!
